# Histogramas de Gradientes Orientados (HOG): uma implementação prática do algoritmo aplicada ao dataset Fashion MNIST

## Introdução à Computação Visual

### André Luiz Moreira Dutra
### Marcos Vinicius Caldeira Pacheco

## Introdução

O problema de detecção de objetos é um dos principais objetos de estudo da computação visual, sendo aplicável a múltiplas áreas, como a identificação facial, por exemplo. Neste contexto, faremos neste trabalho uma implementação própria do algoritmo HOG (Histogramas de Gradientes Orientados) de extração de features de imagens. Em seguida, aplicaremos o algoritmo implementado ao dataset Fashion MNIST para detecção de peças de vestuário, e visualizaremos a disposição espacial das instâncias geradas com os vetores de descritores extraídos.

## O algoritmo HOG

O algoritmo consiste em, após normalizar a imagem, aplicar filtros sobel para extrair as derivadas horizontais e verticais da imagem, e a partir delas extrair a matriz ângulos das derivadas. Em seguida separamos a matriz em blocos quadrados menores e, para cada bloco, fazemos histogramas dos intervalos de ângulos mais frequentes. A sequência de valores de todos os histogramas extraídos de todos os blocos consitui o vetore de descritores HOG, que pode ser utilizado, por exemplo, para o desenvolvimento de um modelo supervisionado de identificação de imagens.

## Implementação

Para a execução correta do código, é necessário ter todas as seguintes bibliotecas instaladas:

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from math import floor
from skimage import filters
from skimage.feature import hog
import os
from sklearn.manifold import TSNE
import plotly.express as px
import pandas as pd

In [2]:
#Retorna um gradiente resultante de uma imagem. Se v==True, então é o gradiente vertical, caso contrário, é o gradiente horizontal.
def getGradient(image, v=True):
    if v:
        grad = filters.sobel_v(image)
    else:
        grad = filters.sobel_h(image)
    return grad

In [ ]:
#A partir dos gradientes vertical e horizontal, retorna a magnitude
def getMagnitude(h, v):
    h2 = h**2
    v2 = v**2
    return (h2+v2)**0.5

In [ ]:
#A partir dos gradientes vertical e horizontal, retorna a orientação (ou matriz de direção)
def getDirection(h, v):
    direct = np.arctan(v/(h+0.00000001))
    direct = np.rad2deg(direct)
    return direct%180

In [ ]:
#A partir dos parâmetros do HOG, retorna uma lista com as cordenadas iniciais de todos os blocos
def getBlocks(image, w, h, s):
    x = image.shape[0]
    y = image.shape[1]
    res = []
    for i in np.arange(0, (y - h + 1), s):
        for j in np.arange(0, (x - w + 1), s):
            res.append((int(i/s), int(j/s)))
    return res

In [ ]:
#Retorna o histograma a partir da magnitude e direção, para determinados bins
def getHist(mag, direct, bins):
    hist = list(np.zeros(len(bins)))
    delta = 180/len(bins)
    for i in range(mag.shape[0]):
        for j in range(mag.shape[1]):
            k = floor(direct[i][j]/delta)
            hist[k] += ((bins[k]+delta - direct[i][j])/delta)*mag[i][j]
            if(k<8):
                hist[k+1] += ((direct[i][j] - bins[k])/delta)*mag[i][j]
    return hist

In [1]:
#Retorna o descritor de um bloco específico a partir dos histogramas
def getDescritor(block, hists):
    x = block[0]
    y = block[1]
    desc = np.array(list(hists[x][y]) + list(hists[x+1][y]) + list(hists[x][y+1]) + list(hists[x+1][y+1]))
    k = (np.sum(desc**2))**0.5
    return list(desc/(k+0.00000001))

In [3]:
#É o HOG propriamente implementado, retorna a descrição da imagem de entrada e os histogramas obtidos
def HOGImage(image, w, h, s):
    horiz = getGradient(image, True)
    verti = getGradient(image, False)

    mag = getMagnitude(horiz, verti)
    direct = getDirection(horiz,verti)%180
    
    bins = [0,20,40,60,80,100,120,140,160]
    hists = np.zeros((floor(image.shape[0]/s),floor(image.shape[1]/s),len(bins)))
    for i in range(hists.shape[0]):
        for j in range(hists.shape[1]):
            hists[i][j] = getHist(mag[i*s:(i+1)*s,j*s:(j+1)*s], direct[i*s:(i+1)*s,j*s:(j+1)*s], bins)
    
    blocks = getBlocks(image, w, h, s)
    descs = []
    for b in blocks:
        descs += getDescritor(b, hists)
    descs = np.around(descs, decimals=8)

    return descs, hists

## Aplicando o algoritmo à base

Vamos agora visualizar o algoritmo na base de dados selecionada. A base de dados escolhida, Fashion MNIST, consiste em uma amostra uma base de dados Fashion MNIST, contendo imagens de artigos de vestuário. Para cada uma das 10 categorias (bag, trouser, coat, dress, etc.) a amostra contém 100 exemplos.